# Tugas Besar Bagian A

# Tugas Besar Pembelajaran Mesin Bagian A
Kelompok 14

**Tentang Program**
Program dalam bahasa Python untuk membuat jaringan saraf tiruan pada bagian feed forward neural network (FFNN). 


**Pembagian Tugas**
| NIM     | Pembagian Tugas                            |
|---------|--------------------------------------------|
| 13521060 | JSON dan Visualisasi Model                 |
| 13521089 | Model dan Abstraksi                        |
| 13521093 | Teknis untuk Komputasi Aktivation Function |
| 13521171 | Laporan dan Test SSE                       |

In [362]:
import json
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class TestCase:
    expected_output: List[List[float]]
    input_size: int
    layers: List[Tuple[int, str]]
    weights: List[np.array]
    input_data: np.array
    max_sse: float

    def print_info(self):
        print(f"Layers\n{self.layers}\n\nWeights{self.weights}\n\n")
        print(f"Input size\n{self.input_size}\n\nInput data\n{self.input_data}\n\n")
        print(f"Expected output\n{self.expected_output}\n\nMax SSE\n{self.max_sse}")

    @staticmethod
    def from_file(filename: str):
        json_read = json.load(open(filename, "r"))
        max_sse = json_read["expect"]["max_sse"]
        output = json_read["expect"]["output"]
        input_size = json_read["case"]["model"]["input_size"]
        layers_raw = json_read["case"]["model"]["layers"]
        layers = []
        for element in layers_raw:
            layers.append(tuple([element["number_of_neurons"], element["activation_function"]]))

        weights_raw = json_read["case"]["weights"]
        weights = []

        for weight_raw in weights_raw:
            weight = []
            for j in weight_raw:
                weight.append(np.array(j))
            weights.append(np.array(weight).T)

        input_data = np.array(json_read["case"]["input"])

        return TestCase(
            expected_output=output,
            input_size=input_size,
            layers=layers,
            weights=weights,
            input_data=input_data,
            max_sse=max_sse
        )

In [363]:
import graphviz as gv

class Neuron:
    weight: np.array

    def __init__(self, weight: np.array):
        self.weight = weight

    """
    return bias + wt•x
    """
    def compute(self, input_data: np.array):
        return self.weight[0] + np.dot(input_data, self.weight[1:])

class Layer:
    neurons: List[Neuron]
    activation_function_name: str
    activation_functions = {
        "linear": lambda x: x,
        "relu": lambda x: np.maximum(0, x),
        "sigmoid": lambda x: 1/(1+np.exp(-x)),
        "softmax": lambda x: np.exp(x)/np.sum(np.exp(x))
    }

    def __init__(self, n_neurons: int, activation_function_name: str, weights: List[np.array]):
        self.neurons = []
        # Initialize neurons in the layer
        for i in range(0, n_neurons):
            self.neurons.append(Neuron(weights[i]))
        self.activation_function_name = activation_function_name

    def predict(self, input_data: np.array) -> np.array:
        raw_result = np.array([neuron.compute(input_data) for neuron in self.neurons])
        return self.activation_functions[self.activation_function_name](raw_result)
            
           

class Model:
    input_size: int
    layers: List[Layer]

    @staticmethod
    def from_test_case(test_case: TestCase):
        return Model(test_case.input_size, test_case.layers, test_case.weights)

    def __init__(self, input_size: int, layers_attr: List[Tuple[int, str]], weights: List[np.array]):
        self.input_size = input_size
        # Initialize layers
        self.layers = []
        for i in range(0, len(layers_attr)):
            layer: Layer = Layer(layers_attr[i][0], layers_attr[i][1], weights[i])
            self.layers.append(layer)
    
    def _predict(self, input_data: np.array) -> np.array:
        layer: Layer = self.layers[0]
        temp_array = layer.predict(input_data)
        for i in range(1, len(self.layers)):
            layer: Layer = self.layers[i]
            temp_array = layer.predict(temp_array)
        
        return temp_array
        
    def predict_batch(self, input_data: List[np.array]) -> List[np.array]:
        # Batch output
        final_output = []
        for i in input_data:
            final_output.append(self._predict(i))
        return final_output

    def visualize(self):
        graph = gv.Digraph(filename="./output/graph.gv")
        for i in range(self.input_size):
            graph.node(f"IN{i}", f"Input Neuron-{i + 1}")
        graph.render()

    def visualize_print(self):
        for i, layer in enumerate(self.layers):
            print(f"/* Hidden Layer-{i + 1} ({layer.activation_function_name}) */")
            for j, neuron in enumerate(layer.neurons):
                print(f"/* Neuron-{j + 1} */")
                if i == 0:
                    source_layer = "Input Layer"
                else:
                    source_layer = f"Hidden Layer-{i}"
                for k, w in enumerate(neuron.weight):
                    if k == 0:
                        continue
                    if w != 0:
                        print(f"{source_layer} Neuron-{k} -> Hidden Layer-{i + 1} Neuron-{j+1} ({w})")
                print("\n\n")
            print("\n\n")

def calculate_sse(output_data: List[np.array], expected_data: List[np.array]):
    if len(output_data) != len(expected_data):
        raise ValueError("Output and Expected Data length doesn't match.")
    
    sses = []

    for output, expected in zip(output_data, expected_data):
        delta = output - expected
        squared_delta = delta ** 2
        sses.append(np.sum(squared_delta))

    return sses

def evaluate_result(output_data: List[np.array], output_reference_data: np.array, expected_data: List[np.array], max_sse: float):
    sses = calculate_sse(output_data, expected_data)

    for i in range(len(output_data)):
        predicted = output_data[i]
        predicted_reference = output_reference_data[i]
        expected = expected_data[i]
        sse = sses[i]

        print(f"Prediction result:\n{predicted}")
        print(f"Prediction reference result:\n{predicted_reference}")
        print(f"Expected output:\n{expected}")

        print(f"sse {sse}\tmax sse{max_sse}")
        print(f"Is below error? {sse < max_sse}")
    

In [364]:
from keras.layers import Dense
from keras.models import Sequential

class ReferenceModel:
    mode: Sequential

    def __init__(self, input_size: int, layers_attr: List[Tuple[int, str]], weights: List[List[np.array]]):
        model = Sequential()

        for (i, (n_neuron, activation)) in enumerate(layers_attr):
            if i == 0:
                model.add(Dense(n_neuron, activation=activation, input_shape=(input_size,)))
            else:
                model.add(Dense(n_neuron, activation=activation))
            
        model.compile()

        for i in range(len(layers_attr)):
            layer = model.layers[i]

            main_weight = np.array([each[1:] for each in weights[i]])
            bias = np.array([each[0] for each in weights[i]])

            layer.set_weights([main_weight.T, bias])
            
        self.model = model

    @staticmethod
    def from_test_case(test_case: TestCase):
        return ReferenceModel(test_case.input_size, test_case.layers, test_case.weights)

# Linear Test Case
Test Case menggunakan linear.json dari test case bagian A Asisten.

In [365]:
case = "linear.json"

print(f"Testing {case}\n\n")
test_case = TestCase.from_file(case)
model = Model.from_test_case(test_case)
model.visualize_print()

reference_model = ReferenceModel.from_test_case(test_case)

reference_prediction = reference_model.model.predict(test_case.input_data)
prediction = model.predict_batch(test_case.input_data)

evaluate_result(prediction, reference_prediction, test_case.expected_output, test_case.max_sse)

Testing linear.json


/* Hidden Layer-1 (linear) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (3.0)






1/1 [==============================] - 0s 380ms/step
Prediction result:
[-11.]
Prediction reference result:
[-11.]
Expected output:
[-11]
sse 0.0	max sse1e-12
Is below error? True
Prediction result:
[-8.]
Prediction reference result:
[-8.]
Expected output:
[-8]
sse 0.0	max sse1e-12
Is below error? True
Prediction result:
[-5.]
Prediction reference result:
[-5.]
Expected output:
[-5]
sse 0.0	max sse1e-12
Is below error? True
Prediction result:
[-2.]
Prediction reference result:
[-2.]
Expected output:
[-2]
sse 0.0	max sse1e-12
Is below error? True
Prediction result:
[1.]
Prediction reference result:
[1.]
Expected output:
[1]
sse 0.0	max sse1e-12
Is below error? True
Prediction result:
[4.]
Prediction reference result:
[4.]
Expected output:
[4]
sse 0.0	max sse1e-12
Is below error? True
Prediction result:
[7.]
Prediction reference result:
[7.]
Expected output:
[7]


# Multilayer Test Case
Test Case menggunakan multilayer.json dari test case bagian A Asisten.

In [366]:
case = "multilayer.json"

print(f"Testing {case}\n\n")
test_case = TestCase.from_file(case)
model = Model.from_test_case(test_case)
model.visualize_print()

reference_model = ReferenceModel.from_test_case(test_case)

reference_prediction = reference_model.model.predict(test_case.input_data)
prediction = model.predict_batch(test_case.input_data)

evaluate_result(prediction, reference_prediction, test_case.expected_output, test_case.max_sse)

Testing multilayer.json


/* Hidden Layer-1 (relu) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (-0.5)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-1 (0.9)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-1 (1.3)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (0.6)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-2 (1.0)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-2 (1.4)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (0.7)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-3 (-1.1)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-3 (1.5)



/* Neuron-4 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-4 (0.5)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-4 (-1.0)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-4 (0.1)






/* Hidden Layer-2 (relu) */
/* Neuron-1 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-1 (-0.4)
Hidden Layer-1 Neuron-2 -> Hidden Layer-2 Neuron-1 (0.7)
Hidden Layer-1 Neuron-3 -> Hidden Layer-2 Neuron-1 (0.2)
Hidde

# Multilayer_Softmax Test Case
Test Case menggunakan multilayer.json dari test case bagian A Asisten.

In [367]:
case = "multilayer_softmax.json"

print(f"Testing {case}\n\n")
test_case = TestCase.from_file(case)
model = Model.from_test_case(test_case)
model.visualize_print()

reference_model = ReferenceModel.from_test_case(test_case)

reference_prediction = reference_model.model.predict(test_case.input_data)
prediction = model.predict_batch(test_case.input_data)

evaluate_result(prediction, reference_prediction, test_case.expected_output, test_case.max_sse)

Testing multilayer_softmax.json


/* Hidden Layer-1 (relu) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.8)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-1 (0.3)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-1 (1.1)
Input Layer Neuron-4 -> Hidden Layer-1 Neuron-1 (0.5)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (-0.7)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-2 (-1.4)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-2 (-1.3)
Input Layer Neuron-4 -> Hidden Layer-1 Neuron-2 (-0.8)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (1.1)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-3 (0.7)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-3 (0.9)
Input Layer Neuron-4 -> Hidden Layer-1 Neuron-3 (1.4)



/* Neuron-4 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-4 (-1.2)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-4 (1.2)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-4 (0.4)
Input Layer Neuron-4 -> Hidden Layer-1 Neuron-4 (-0.9)

# Relu Test Case
Test Case menggunakan relu.json dari test case bagian A Asisten.

In [368]:
case = "relu.json"

print(f"Testing {case}\n\n")
test_case = TestCase.from_file(case)
model = Model.from_test_case(test_case)
model.visualize_print()

reference_model = ReferenceModel.from_test_case(test_case)

reference_prediction = reference_model.model.predict(test_case.input_data)
prediction = model.predict_batch(test_case.input_data)

evaluate_result(prediction, reference_prediction, test_case.expected_output, test_case.max_sse)

Testing relu.json


/* Hidden Layer-1 (relu) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.47)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-1 (1.1)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (-0.6)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-2 (-1.3)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (0.2)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-3 (0.5)






1/1 [==============================] - 0s 339ms/step
Prediction result:
[0.31  0.    0.375]
Prediction reference result:
[0.30999997 0.         0.37500003]
Expected output:
[0.31, 0, 0.375]
sse 3.0814879110195774e-33	max sse1e-06
Is below error? True


# Sigmoid Test Case
Test Case menggunakan sigmoid.json dari test case bagian A Asisten.

In [369]:
case = "sigmoid.json"

print(f"Testing {case}\n\n")
test_case = TestCase.from_file(case)
model = Model.from_test_case(test_case)
model.visualize_print()

reference_model = ReferenceModel.from_test_case(test_case)

reference_prediction = reference_model.model.predict(test_case.input_data)
prediction = model.predict_batch(test_case.input_data)

evaluate_result(prediction, reference_prediction, test_case.expected_output, test_case.max_sse)

Testing sigmoid.json


/* Hidden Layer-1 (sigmoid) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (-1.2)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-1 (1.4)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-1 (-0.7)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (-1.7)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-2 (-1.6)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-2 (1.1)






/* Hidden Layer-2 (sigmoid) */
/* Neuron-1 */
Hidden Layer-1 Neuron-2 -> Hidden Layer-2 Neuron-1 (2.1)



/* Neuron-2 */
Hidden Layer-1 Neuron-2 -> Hidden Layer-2 Neuron-2 (-0.2)



/* Neuron-3 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-3 (-1.5)



/* Neuron-4 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-4 (0.7)
Hidden Layer-1 Neuron-2 -> Hidden Layer-2 Neuron-4 (1.8)






1/1 [==============================] - 0s 460ms/step
Prediction result:
[0.41197346 0.8314294  0.53018536 0.31607396]
Prediction reference result:
[0.41197345 0.8314294  0.53018534 0.316

# Softmax Test Case
Test Case menggunakan softmax.json dari test case bagian A Asisten.

In [370]:
case = "softmax.json"

print(f"Testing {case}\n\n")
test_case = TestCase.from_file(case)
model = Model.from_test_case(test_case)
model.visualize_print()

reference_model = ReferenceModel.from_test_case(test_case)

reference_prediction = reference_model.model.predict(test_case.input_data)
prediction = model.predict_batch(test_case.input_data)

evaluate_result(prediction, reference_prediction, test_case.expected_output, test_case.max_sse)

Testing softmax.json


/* Hidden Layer-1 (softmax) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (-0.2)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-1 (0.3)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-1 (0.4)
Input Layer Neuron-4 -> Hidden Layer-1 Neuron-1 (0.5)
Input Layer Neuron-5 -> Hidden Layer-1 Neuron-1 (-0.6)
Input Layer Neuron-6 -> Hidden Layer-1 Neuron-1 (-0.7)
Input Layer Neuron-7 -> Hidden Layer-1 Neuron-1 (0.8)
Input Layer Neuron-8 -> Hidden Layer-1 Neuron-1 (0.9)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (0.8)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-2 (-0.7)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-2 (0.6)
Input Layer Neuron-4 -> Hidden Layer-1 Neuron-2 (0.5)
Input Layer Neuron-5 -> Hidden Layer-1 Neuron-2 (0.4)
Input Layer Neuron-6 -> Hidden Layer-1 Neuron-2 (-0.3)
Input Layer Neuron-7 -> Hidden Layer-1 Neuron-2 (0.2)
Input Layer Neuron-8 -> Hidden Layer-1 Neuron-2 (-0.1)



/* Neuron-3 */
Input Layer Neuron-1 -> H

# Tugas Besar Bagian B

# Tugas Besar Pembelajaran Mesin Bagian B
Kelompok 14

**Tentang Program**

Program dalam bahasa Python untuk membuat neural network dengan algoritma Backpropagation. 


**Pembagian Tugas**
| NIM     | Pembagian Tugas                            |
|---------|--------------------------------------------|
| 13521060 | Pembuatan Model                           |
| 13521089 | Pembuatan Model, Penjelasan Algoritma     |
| 13521093 | Pembuatan Model dan Algoritma |
| 13521171 | Laporan dan Testing                    |

## Setup Program
Bagian ini mengimpor library, dan setup konfigurasi tipe program. 

***DEBUG_MODE*** : Mode

***TESTCASE_MODE***: Mode menjalankan testcase backprogagation

***TESTCASE_REFERENCE_MODE***: Mode menjalankan testcase dengan library

In [371]:
import tensorflow as tf
from typing import List
from dataclasses import dataclass
from tensorflow.keras.layers import Dense, Input
from keras.models import Sequential
from keras.optimizers import SGD
from keras.losses import MeanSquaredError
from tensorflow.keras.models import Model

import numpy as np
import json
import math

In [372]:
# Setup

np.random.seed(0)

DEBUG_MODE: bool = False
TESTCASE_MODE: bool = True
TESTCASE_REFERENCE_MODE: bool = True

def run_test_case_mode(filename):
    if TESTCASE_MODE:
        run_test_case(filename)

def run_test_case_reference_mode(filename):
    if TESTCASE_REFERENCE_MODE:
        run_test_on_reference(filename)

def print_debug(value: str):
    if DEBUG_MODE:
        print(value)

## Metode Fungsi Aktivasi
Berikut merupakan setup untuk metode fungsi aktivasi dan turunannya.

In [373]:
class Methods:
    def __init__(self):
        self.activation_functions = {
            "linear": self.activation_linear,
            "relu": self.activation_relu,
            "sigmoid": self.activation_sigmoid,
            "softmax": self.activation_softmax
        }

        self.derived_activation_functions = {
            "linear": self.derivation_linear,
            "relu": self.derivation_relu,
            "sigmoid": self.derivation_sigmoid,
            "softmax": self.derivation_softmax
        }
        
        self.loss_functions = {
            "linear": self.sse,
            "relu": self.sse,
            "sigmoid": self.sse,
            "softmax": self.softmax_loss
        }

    def activation_linear(self, x: np.array):
        return x

    def activation_relu(self, x: np.array):
        return np.maximum(0, x)

    def activation_sigmoid(self, x: np.array):
        return 1/(1+np.exp(-x, dtype=np.float64))

    def activation_softmax(self, x: np.array):
        return np.exp(x, dtype=np.float64)/np.sum(np.exp(x), dtype=np.float64)

    def derivation_linear(self, x: np.array):
        return np.ones(len(x), dtype=np.float64)

    def derivation_relu(self, x: np.array):
        return 1. * (x > 0)
    
    def derivation_sigmoid(self, x: np.array):
        sigm = self.activation_sigmoid(x)
        return sigm*(1-sigm)

    def derivation_softmax(self, x: np.array, y: np.array):
        return self.activation_softmax(x) - y

    def sse(self, o: np.array, t: np.array):
        return 0.5*np.sum((t-o)**2, dtype=np.float64)

    def softmax_loss(self, o: np.array, t: np.array):
        return np.sum(-t*np.log(o))

    

## Konfigurasi Model
Pada cell berikut merupakan konfigurasi model untuk algoritma backpropagation.

### Kelas Neuron
Kelas ini mewakili satu neuron dalam neural network. Disini, mencakup properti untuk bobot dan bias neuron serta metode untuk menghitung nilai output (nilai-z), memperbarui bobot, dan mempertahankan pembaruan.

In [374]:
class Neuron:
    weight: np.array
    weight_size: int
    d_weight: np.array
    d_bias: np.array
    z_val: float
    bias: float

    def __init__(self, weight_size: int):
        self.weight_size = weight_size
        self.weight = np.random.rand(weight_size)
        self.bias = np.random.rand(1)[0]

    """ Return z value

    z = bias + wt•x
    """
    def compute_z(self, input_data: np.array) -> float:
        self.z_val = self.bias + np.dot(input_data, self.weight)
        return self.z_val

    def set_weight(self, weight: np.array, bias: float):
        if len(weight) != self.weight_size:
            raise Exception(f"Weight size not match expect {self.weight_size} got {len(weight)}")
        
        self.weight = weight
        self.bias = bias

    def init_d_weight(self, batch_size: int):
        self.d_weight = np.zeros((batch_size, self.weight_size), dtype=np.float64)
        self.d_bias = np.zeros((batch_size, ), dtype=np.float64)

    def persist_d_weight(self, learning_rate: float):
        self.weight -= learning_rate * self.d_weight.mean(axis=0, dtype=np.float64)
        print_debug(f"\t\tb -= n * d_bias = {learning_rate} * {self.d_bias.mean(axis=0, dtype=np.float64)}")
        self.bias -= learning_rate * self.d_bias.mean(dtype=np.float64)
        self.d_weight = None
        self.d_bias = None
        

### Kelas Layer
Kelas ini mewakili layer dari neural network yang terdiri dari beberapa neuron. Disini dikelola operasi yang terkait dengan sekelompok neuron, misalnya mengelola turunan untuk learning.

In [375]:
class Layer:
    activation: str
    input_size: int
    neurons: List[Neuron]
    activation_function: str
    methods: Methods
    n_neurons: int

    result_raw: np.array # dimension matched with number of neurons in the layer
    result: np.array # dimension matched with number of neurons in the layer
    result_derivative: np.array # dimension matched with number of neurons in the layer
    do_c_do_a: np.array # dimension matched with number of neurons in the layer
    
    def __init__(self,
                 n_neurons: int,
                 activation_function: str,
                 input_size: int = None):
        self.input_size = input_size
        self.activation_function = activation_function
        self.n_neurons = n_neurons
        self.neurons = []
        self.methods = Methods()

    def init_neurons(self, n_weights: int = None):
        if self.input_size is not None:
            self.neurons = [Neuron(self.input_size) for _ in range(self.n_neurons)]
        elif n_weights is None:
            raise Exception("n_weights should not be none for noninput layer")
        else:
            self.neurons = [Neuron(n_weights) for _ in range(self.n_neurons)]

    def update_weights(self, weights: np.array, bias: np.array):
        for i, neuron in enumerate(self.neurons):
            neuron.set_weight(weights[i], bias[i])

    def update_weights_from_normalized(self, weights: np.array):
        self.update_weights(weights[1:].T, weights[0])

    def predict(self, input_data: np.array) -> np.array:
        self.result_raw = np.array([neuron.compute_z(input_data) for neuron in self.neurons])
        #print_debug(f"z: {self.result_raw}, activation function: {self.activation_function}")
        self.result = self.methods.activation_functions[self.activation_function](self.result_raw)
        print_debug(f"a: {self.result}")
        return self.result
    
    def predict_derivative(self, target: np.array = None):
        if self.activation_function == 'softmax':
            self.result_derivative = self.methods.derived_activation_functions[self.activation_function](self.result_raw, target)
        else:
            self.result_derivative = self.methods.derived_activation_functions[self.activation_function](self.result_raw)

        return self.result_derivative
    
    def persist_d_weight(self, learning_rate: float):
        for i, neuron in enumerate(self.neurons):
            print_debug(f"Persisting for neuron-{i}")
            print_debug(f"\tBefore: {neuron.bias} {neuron.weight}")
            neuron.persist_d_weight(learning_rate)
            print_debug(f"\tAfter: {neuron.bias} {neuron.weight}")

    def init_d_weight(self, batch_size: int):
        for neuron in self.neurons:
            neuron.init_d_weight(batch_size)

    def clear_prev_result(self):
        self.result_raw = None
        self.result = None
        self.result_derivative = None
        self.do_c_do_a = None

    @property
    def normalized_weights(self) -> np.array:
        return np.array([np.append(np.array([n.bias], dtype=np.float64), n.weight) for n in self.neurons], dtype=np.float64).T

### Kelas SavedLayer dan SavedModel
Kelas ini merupakan kelas bantuan untuk menyimpan  pembaruan layer dan pembaruan model

In [376]:
@dataclass
class SavedLayer:
    activation_function: str
    n_neuron: int
    # three dimensional array
    # first dimension for every neuron
    # second dimension consist of two element
    # the first one is the bias value
    # the second one is the weights value
    weights: np.array

    def to_dict(self):
        return {
            "activation_function": self.activation_function,
            "n_neuron": self.n_neuron,
            "weights": self.weights.tolist()
        }

@dataclass
class SavedModel:
    layers: List[SavedLayer]
    input_size: int

    @staticmethod
    def from_file(filename: str):
        f = open(filename, "r")
        json_read = json.loads(f.read())
        f.close()

        me = SavedModel(layers=[], input_size=json_read["input_size"])

        for layer in json_read["layers"]:
            me.layers.append(SavedLayer(
                activation_function=layer["activation_function"],
                n_neuron=layer["n_neuron"],
                weights=np.array(layer["weights"])
            ))

        me.input_size = json_read["input_size"]
        
        return me
    
    @staticmethod
    def from_model(model: Model):
        me = SavedModel(layers=[], input_size=model.layers[0].input_size)

        for layer in model.layers:
            me.layers.append(SavedLayer(
                activation_function=layer.activation_function,
                n_neuron=layer.n_neurons,
                weights=layer.normalized_weights
            ))

        return me
    
    def save_to(self, filename: str):
        layers_dict = [each.to_dict() for each in self.layers]

        with open(filename, "w") as f:
            json.dump({
                "layers": layers_dict,
                "input_size": self.input_size
            }, f)

### Kelas Model
Kelas ini mewakili model dari neural network yang terdiri dari beberapa neuron. Disini dikelola operasi yang terkait dengan sekelompok layer yang membentuk model, misalnya mengelola weight pada model, visualisasikan model, dan memprediksi hasil training.

In [377]:
class Model:
    layers: List[Layer]
    training_stop_reason: str

    def __init__(self):
        self.layers = []
        pass

    @staticmethod
    def from_saved_model(saved_model: SavedModel):
        model = Model()

        for i, layer in enumerate(saved_model.layers):
            if i == 0:
                model.add(
                    Layer(
                        n_neurons=layer.n_neuron,
                        activation_function=layer.activation_function,
                        input_size=saved_model.input_size
                    )
                )
            else:
                model.add(
                    Layer(
                        n_neurons=layer.n_neuron,
                        activation_function=layer.activation_function,
                    )
                )
        
        model.compile()

        for i, layer in enumerate(model.layers):
            layer.update_weights_from_normalized(saved_model.layers[i].weights)

        return model

    def add(self, layer: Layer):
        self.layers.append(layer)

    """ Initialize empty weights
    """
    def compile(self):
        n_weights: int = None
        n_layer = len(self.layers)

        for i, layer in enumerate(self.layers):
            if i == 0:
                layer.init_neurons()
            else:
                layer.init_neurons(n_weights)

            if i != n_layer - 1 and layer.activation_function == 'softmax':
                raise Exception("Cannot use softmax activation function in hidden layer")
            
            n_weights = layer.n_neurons
    
    def update_weights(self, weights: List[np.array], bias: List[np.array]):
        for i, layer in enumerate(self.layers):
            layer.update_weights(weights[i], bias[i])

    def clear_prev_result(self):
        for layer in self.layers:
            layer.clear_prev_result()

    def fit(self,
            x: np.array,
            y: np.array,
            learning_rate: float = 0.1,
            epochs: int = 5,
            batch_size: int = 5,
            error_threshold: float = 0
            ):
        
        current_error = np.inf
        epoch = 0

        print("Begin training model\n\n")

        while current_error > error_threshold and epoch < epochs:
            print(f"Epoch {epoch}")

            error = 0

            for i, x_batch, y_batch in self._generate_mini_batches(x, y, batch_size):
                print_debug(f"\n----------------------\nTraining for batch-{i} with size {len(x_batch)}")
                error += self._backpropagate(x_batch, y_batch, learning_rate)
    
            current_error = error / math.ceil(len(x)/batch_size)
                
            print(f"training loss {current_error}")
            epoch += 1
            print("\n")

        if current_error <= error_threshold:
            self.training_stop_reason = "error_threshold"
        elif epoch >= epochs:
            self.training_stop_reason = "max_iteration"
            

    def _init_d_weight(self, batch_size: int):
        for layer in self.layers:
            layer.init_d_weight(batch_size)

    def _persist_d_weight(self, learning_rate: float):
        for i, layer in enumerate(self.layers):
            print_debug(f"Persisting d_weight for layer-{i}")
            layer.persist_d_weight(learning_rate)

    def _predict(self, input_data: np.array, with_derivative: bool = False, target: np.array = None) -> np.array:
        layer: Layer = self.layers[0]
        temp_array = layer.predict(input_data)
        if with_derivative:
            layer.predict_derivative(target)
            
        for i in range(1, len(self.layers)):
            layer: Layer = self.layers[i]
            temp_array = layer.predict(temp_array)
            if with_derivative:
                layer.predict_derivative(target)
            print_debug(f"prediction layer {i}: {temp_array}")
        
        return temp_array
    
    def predict_batch(self, input_data: List[np.array]) -> List[np.array]:
        # Batch output
        final_output = []
        for data in input_data:
            final_output.append(self._predict(data))
        return final_output
    
    def visualize(self):
        for i, layer in enumerate(self.layers):
            print(f"/* Layer-{i + 1} ({layer.activation_function}) */")
            for j, neuron in enumerate(layer.neurons):
                print(f"/* Neuron-{j + 1} */")
                if i == 0:
                    source_layer = "Input Layer"
                else:
                    source_layer = f"Hidden Layer-{i}"
                for k, w in enumerate(neuron.weight):
                    if k == 0:
                        continue
                    if w != 0:
                        print(f"{source_layer} Neuron-{k} -> Hidden Layer-{i + 1} Neuron-{j+1} ({w})")
                print("\n\n")
            print("\n\n")

    def _generate_mini_batches(self, x: np.array, y: np.array, batch_size: int):
        start_index = 0
        batch_number = 0
        input_size = len(x)

        while start_index < input_size:
            end_index = min(start_index + batch_size, input_size)
            
            yield batch_number, x[start_index:end_index], y[start_index:end_index]

            start_index += batch_size
            batch_number += 1

    def _backpropagate(self, 
                       x: np.array, 
                       y: np.array, 
                       learning_rate: float):
        
        self._init_d_weight(len(x))

        for nth_element, element in enumerate(x):
            print_debug(f"----------------------\nBackpropagating for element {nth_element} in batch")

            if self.layers[-1].activation_function == 'softmax':
                self._predict(x[nth_element], with_derivative=True, target=y[nth_element])
            else:
                self._predict(x[nth_element], with_derivative=True)

            for i in range(len(self.layers)-1, -1, -1):
                print_debug(f"layer current: {i}")
                layer_current = self.layers[i]
                layer_previous = None if i == 0 else self.layers[i - 1]
                layer_next = None if i == len(self.layers) - 1 else self.layers[i + 1]

                a_val_prev = layer_previous.result if layer_previous is not None else element # sepanjang input/ neuron sebelumnya

                if layer_next is not None:
                    # Update do_c_do_a for hidden layer

                    # Calculate do_c_do_a to be used in calculating do_c_do_w
                    # do_c_do_a of output layer has a special value, calculated separately below
                    # this is for when layer next is hidden layer
                    layer_current.do_c_do_a = np.zeros((layer_current.n_neurons, 1))
                    for k, neuron_current in enumerate(layer_current.neurons):
                        for j, neuron_next in enumerate(layer_next.neurons):
                            layer_current.do_c_do_a[k] += neuron_next.weight[k] * layer_next.result_derivative[j] * layer_next.do_c_do_a[j]
                            
                else:
                    # Update do_c_do_a for output layer
                    layer_current.do_c_do_a = 2 * (layer_current.result - y[nth_element]) # dimension same as n_neuron of that layer
                #print_debug(f"layer current doC / doA: {layer_current.do_c_do_a}")
                # Update weight the layer
                for j, neuron_current in enumerate(layer_current.neurons): 
                    if layer_next is None and layer_current.activation_function == 'softmax':
                        do_c_do_w = a_val_prev * layer_current.do_c_do_a[j] * 0.5
                        neuron_current.d_bias[nth_element] = layer_current.do_c_do_a[j] * 0.5
                    else:
                        do_c_do_w = a_val_prev * layer_current.result_derivative[j] * layer_current.do_c_do_a[j]
                        neuron_current.d_bias[nth_element] = layer_current.result_derivative[j] * layer_current.do_c_do_a[j]
                    neuron_current.d_weight[nth_element] = do_c_do_w
                    #print_debug(f"doC / doW = {a_val_prev} * {layer_current.result_derivative[j]} * {layer_current.do_c_do_a[j]} = {do_c_do_w}")
            self.clear_prev_result()

        self._persist_d_weight(learning_rate)
        return Methods().loss_functions[self.layers[-1].activation_function](self.predict_batch(x), y)

## Konfigurasi Running Testcase
Berikut fungsi untuk menjalankan testcase

In [378]:
def run_test_case(filename: str):
    print(f"Running test case for {filename}\n\n")

    f = open(filename, "r")
    data = json.loads(f.read())
    f.close()

    n_layer = len(data["case"]["model"]["layers"])

    model = Model()

    for i in range(n_layer):
        layer = data["case"]["model"]["layers"][i]
        if i == 0:
            model.add(
                Layer(
                    n_neurons=layer["number_of_neurons"],
                    activation_function=layer["activation_function"],
                    input_size=data["case"]["model"]["input_size"]
                )
            )
        else:
            model.add(
                Layer(
                    n_neurons=layer["number_of_neurons"],
                    activation_function=layer["activation_function"],
                )
            )

    model.compile()

    for i in range(n_layer):
        model.layers[i].update_weights_from_normalized(np.array(data["case"]["initial_weights"][i]))
        print(f"Initial weight for layer {i}\n{model.layers[i].normalized_weights}")

    model.visualize()

    data_input = np.array(data["case"]["input"])
    data_expect = np.array(data["case"]["target"])

    print(f"Training data input {len(data['case']['input'])} rows")

    # begin training
    model.fit(
        x=data_input,
        y=data_expect,
        learning_rate=data["case"]["learning_parameters"]["learning_rate"],
        epochs=data["case"]["learning_parameters"]["max_iteration"],
        batch_size=data["case"]["learning_parameters"]["batch_size"],
        error_threshold=data["case"]["learning_parameters"]["error_threshold"]
    )

    print(f"\nStop reason by {model.training_stop_reason} expected {data['expect']['stopped_by']}\n")

    for i in range(n_layer):
        expect_weight = np.array(data["expect"]["final_weights"][i])
        result_weight = model.layers[i].normalized_weights
        print(f"result weight for layer-{i}\n{result_weight}")
        print(f"expected weight for layer-{i}\n{expect_weight}")
        print(f"abs weight difference for layer-{i}\n{np.abs(result_weight - expect_weight)}")
        rmse = np.mean((expect_weight-result_weight)**2, dtype=np.float64)
        print(f"\nLayer {i} weights rmse {rmse}\n")

    output = model.predict_batch(np.array(data["case"]["input"]))

    print(f"output {output}\nexpect {data_expect}")

    error_result = np.mean((output-data_expect)**2)
    print(f"Testing result error {error_result}")

In [379]:
def run_test_on_reference(filename: str):
    print(f"Running test case for {filename}\n\n")

    f = open(filename, "r")
    data = json.loads(f.read())
    f.close()

    input_layer = Input(shape=(data["case"]["model"]["input_size"],))

    n_layer = len(data["case"]["model"]["layers"])

    for i in range(n_layer):
        layer = data["case"]["model"]["layers"][i]
        layers = Dense(
                layer["number_of_neurons"],
                activation=layer["activation_function"],
            )(input_layer if i == 0 else layers)       
    
    model = tf.keras.Model(inputs=input_layer, outputs=layers)    

    model.compile(
        optimizer=SGD(data["case"]["learning_parameters"]["learning_rate"]),
        loss=MeanSquaredError(
            reduction="sum_over_batch_size", 
            name="mean_squared_error"
        )
    )

    for i in range(n_layer):
        weights = data["case"]["initial_weights"][i]
        layer = model.layers[i + 1]
        main_weight = np.array(weights[1:])
        bias = np.array(weights[0])

        layer.set_weights([main_weight, bias])

    data_input = np.array(data["case"]["input"])
    data_expect = np.array(data["case"]["target"])
        
    model.fit(
        x=data_input,
        y=data_expect,
        batch_size=data["case"]["learning_parameters"]["batch_size"],
        epochs=data["case"]["learning_parameters"]["max_iteration"]
    )

    output = model.predict(np.array(data["case"]["input"]))

    for i in range(n_layer):
            layer = model.layers[i + 1]
            raw_weight = layer.get_weights()
            print(f"raw weight\n{raw_weight}\n weight shape {raw_weight[0].shape} bias shape {raw_weight[1].shape}")
            normalized_weight = np.array([raw_weight[1].tolist()] + raw_weight[0].tolist())
            print(f"result weight for layer-{i}\n{normalized_weight}")
            expect_weight = np.array(data["expect"]["final_weights"][i])
            print(f"expected weight for layer-{i}\n{expect_weight}")
            rmse = np.mean((expect_weight-normalized_weight)**2, dtype=np.float64)
            print(f"\nLayer {i} weights rmse {rmse}\n")

    print(f"output \n{output}\nexpect {data_expect}")

    error_result = np.mean((output-data_expect)**2)
    print(f"Testing result error {error_result}")

In [380]:
test_cases = [
    "linear.json", # 0
    "linear_small_lr.json", # 1
    "linear_two_iteration.json", # 2
    "mlp.json", # 3
    "relu_b.json", # 4
    "sigmoid.json", # 5
    "softmax.json", # 6
    "softmax_two_layer.json" # 7
]

### Linear.json
Testcase untuk linear.json

In [381]:
run_test_case_mode(f"tc_b/{test_cases[0]}")

Running test case for tc_b/linear.json


Initial weight for layer 0
[[ 0.1  0.3  0.2]
 [ 0.4  0.2 -0.7]
 [ 0.1 -0.8  0.5]]
/* Layer-1 (linear) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.1)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (-0.8)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (0.5)






Training data input 2 rows
Begin training model


Epoch 0
training loss 0.18184999999999998



Stop reason by max_iteration expected max_iteration

result weight for layer-0
[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]
expected weight for layer-0
[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]
abs weight difference for layer-0
[[2.77555756e-17 0.00000000e+00 1.38777878e-17]
 [1.11022302e-16 5.55111512e-17 0.00000000e+00]
 [5.55111512e-17 0.00000000e+00 0.00000000e+00]]

Layer 0 weights rmse 2.1613213820345645e-33

output [array([ 2.42,  0.56, -2.19]), array([ 1.42, -0.74, -0.04])]
expect [[ 2

In [382]:
run_test_case_reference_mode(f"tc_b/{test_cases[0]}")

Running test case for tc_b/linear.json


1/1 [==============================] - 0s 338ms/step
raw weight
[array([[ 0.48      ,  0.23333333, -0.7633333 ],
       [ 0.16      , -0.7666667 ,  0.45666665]], dtype=float32), array([0.14      , 0.32000002, 0.17      ], dtype=float32)]
 weight shape (2, 3) bias shape (3,)
result weight for layer-0
[[ 0.14        0.32000002  0.17      ]
 [ 0.47999999  0.23333333 -0.76333332]
 [ 0.16       -0.76666671  0.45666665]]
expected weight for layer-0
[[ 0.22  0.36  0.11]
 [ 0.64  0.3  -0.89]
 [ 0.28 -0.7   0.37]]

Layer 0 weights rmse 0.009338272532416321

output 
[[ 1.7399999   0.2533333  -1.6633333 ]
 [ 0.93999994 -0.98        0.32      ]]
expect [[ 2.   0.3 -1.9]
 [ 1.3 -0.7  0.1]]
Testing result error 0.06369816974445938


### Linear_small_lr.json
Testcase untuk Linear_small_lr.json

In [383]:
run_test_case_mode(f"tc_b/{test_cases[1]}")

Running test case for tc_b/linear_small_lr.json


Initial weight for layer 0
[[ 0.1  0.3  0.2]
 [ 0.4  0.2 -0.7]
 [ 0.1 -0.8  0.5]]
/* Layer-1 (linear) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.1)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (-0.8)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (0.5)






Training data input 2 rows
Begin training model


Epoch 0
training loss 0.6462307850000001



Stop reason by max_iteration expected max_iteration

result weight for layer-0
[[ 0.1012  0.3006  0.1991]
 [ 0.4024  0.201  -0.7019]
 [ 0.1018 -0.799   0.4987]]
expected weight for layer-0
[[ 0.1008  0.3006  0.1991]
 [ 0.402   0.201  -0.7019]
 [ 0.101  -0.799   0.4987]]
abs weight difference for layer-0
[[0.0004 0.     0.    ]
 [0.0004 0.     0.    ]
 [0.0008 0.     0.    ]]

Layer 0 weights rmse 1.0666666666666786e-07

output [array([ 1.4102,  0.1046, -1.4079]), array([ 0.7072, -1.0964,  0.4946])]
expect [[ 2.   0.3 -1.9]
 [

In [384]:
run_test_case_reference_mode(f"tc_b/{test_cases[1]}")

Running test case for tc_b/linear_small_lr.json


1/1 [==============================] - 0s 409ms/step
raw weight
[array([[ 0.40080002,  0.20033334, -0.70063335],
       [ 0.1006    , -0.7996667 ,  0.49956667]], dtype=float32), array([0.1004    , 0.30020002, 0.1997    ], dtype=float32)]
 weight shape (2, 3) bias shape (3,)
result weight for layer-0
[[ 0.1004      0.30020002  0.1997    ]
 [ 0.40080002  0.20033334 -0.70063335]
 [ 0.1006     -0.7996667   0.49956667]]
expected weight for layer-0
[[ 0.1008  0.3006  0.1991]
 [ 0.402   0.201  -0.7019]
 [ 0.101  -0.799   0.4987]]

Layer 0 weights rmse 6.138215430143493e-07

output 
[[ 1.4034001   0.10153332 -1.4026334 ]
 [ 0.70239997 -1.0988001   0.4982    ]]
expect [[ 2.   0.3 -1.9]
 [ 1.3 -0.7  0.1]]
Testing result error 0.21957075650185745


### linear_two_iteration.json
Testcase untuk linear_two_iteration.json

In [385]:
run_test_case_mode(f"tc_b/{test_cases[2]}")

Running test case for tc_b/linear_two_iteration.json


Initial weight for layer 0
[[ 0.1  0.3  0.2]
 [ 0.4  0.2 -0.7]
 [ 0.1 -0.8  0.5]]
/* Layer-1 (linear) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.1)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (-0.8)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (0.5)






Training data input 2 rows
Begin training model


Epoch 0
training loss 0.18184999999999998


Epoch 1
training loss 0.05544650000000001



Stop reason by max_iteration expected max_iteration

result weight for layer-0
[[ 0.166  0.338  0.153]
 [ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]]
expected weight for layer-0
[[ 0.166  0.338  0.153]
 [ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]]
abs weight difference for layer-0
[[0.00000000e+00 5.55111512e-17 0.00000000e+00]
 [0.00000000e+00 2.77555756e-17 1.11022302e-16]
 [2.77555756e-17 0.00000000e+00 0.00000000e+00]]

Layer 0 weights rmse 1.8831315011786308e-33

out

In [386]:
run_test_case_reference_mode(f"tc_b/{test_cases[2]}")

Running test case for tc_b/linear_two_iteration.json


Epoch 1/2
1/1 [==============================] - 2s 2s/step - loss: 0.2217
Epoch 2/2
1/1 [==============================] - 0s 472ms/step
raw weight
[array([[ 0.518     ,  0.24733335, -0.79433334],
       [ 0.19266666, -0.74644446,  0.4341111 ]], dtype=float32), array([0.16066667, 0.33088893, 0.15477778], dtype=float32)]
 weight shape (2, 3) bias shape (3,)
result weight for layer-0
[[ 0.16066667  0.33088893  0.15477778]
 [ 0.51800001  0.24733335 -0.79433334]
 [ 0.19266666 -0.74644446  0.43411109]]
expected weight for layer-0
[[ 0.166  0.338  0.153]
 [ 0.502  0.226 -0.789]
 [ 0.214 -0.718  0.427]]

Layer 0 weights rmse 0.00023738832876708866

output 
[[ 1.9073334   0.3264445  -1.794111  ]
 [ 1.064      -0.9146667   0.22866662]]
expect [[ 2.   0.3 -1.9]
 [ 1.3 -0.7  0.1]]
Testing result error 0.02313863068351268


### mlp.json
Testcase untuk mlp.json

In [387]:
run_test_case_mode(f"tc_b/{test_cases[3]}")

Running test case for tc_b/mlp.json


Initial weight for layer 0
[[ 0.1  0.2]
 [-0.3  0.5]
 [ 0.4  0.5]]
Initial weight for layer 1
[[ 0.2  0.1]
 [ 0.4 -0.5]
 [ 0.7  0.8]]
/* Layer-1 (linear) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.4)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (0.5)






/* Layer-2 (relu) */
/* Neuron-1 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-1 (0.7)



/* Neuron-2 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-2 (0.8)






Training data input 2 rows
Begin training model


Epoch 0
training loss 0.330162168871235



Stop reason by max_iteration expected max_iteration

result weight for layer-0
[[ 0.08592   0.32276 ]
 [-0.33872   0.46172 ]
 [ 0.449984  0.440072]]
expected weight for layer-0
[[ 0.08592   0.32276 ]
 [-0.33872   0.46172 ]
 [ 0.449984  0.440072]]
abs weight difference for layer-0
[[1.38777878e-17 5.55111512e-17]
 [0.00000000e+00 0.00000000e+00]
 [5.55111512e-17 0.00000000e+00]]

Layer 0 w

In [388]:
run_test_case_reference_mode(f"tc_b/{test_cases[3]}")

Running test case for tc_b/mlp.json


1/1 [==============================] - 0s 380ms/step
raw weight
[array([[-0.31936002,  0.48086   ],
       [ 0.424992  ,  0.470036  ]], dtype=float32), array([0.09296   , 0.26138002], dtype=float32)]
 weight shape (2, 2) bias shape (2,)
result weight for layer-0
[[ 0.09296     0.26138002]
 [-0.31936002  0.48085999]
 [ 0.424992    0.470036  ]]
expected weight for layer-0
[[ 0.08592   0.32276 ]
 [-0.33872   0.46172 ]
 [ 0.449984  0.440072]]

Layer 0 weights rmse 0.001013442309940077

raw weight
[array([[ 0.417952  , -0.51584   ],
       [ 0.69251996,  0.79120004]], dtype=float32), array([0.2374, 0.144 ], dtype=float32)]
 weight shape (2, 2) bias shape (2,)
result weight for layer-1
[[ 0.2374      0.14399999]
 [ 0.417952   -0.51583999]
 [ 0.69251996  0.79120004]]
expected weight for layer-1
[[ 0.2748    0.188   ]
 [ 0.435904 -0.53168 ]
 [ 0.68504   0.7824  ]]

Layer 1 weights rmse 0.0006735552549592174

output 
[[0.3583628 0.       ]
 [0.        0.259

### RelU_b.json
Testcase untuk relu_b.json

In [389]:
run_test_case_mode(f"tc_b/{test_cases[4]}")

Running test case for tc_b/relu_b.json


Initial weight for layer 0
[[-0.2  0.2  1. ]
 [ 0.3  0.5  0.5]
 [-0.5 -1.   0.5]]
/* Layer-1 (relu) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (-0.5)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (-1.0)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (0.5)






Training data input 2 rows
Begin training model


Epoch 0
training loss 1.7831637775499998



Stop reason by max_iteration expected max_iteration

result weight for layer-0
[[-0.211   0.105   0.885 ]
 [ 0.3033  0.5285  0.3005]
 [-0.489  -0.905   0.291 ]]
expected weight for layer-0
[[-0.211   0.105   0.885 ]
 [ 0.3033  0.5285  0.3005]
 [-0.489  -0.905   0.291 ]]
abs weight difference for layer-0
[[2.77555756e-17 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.55111512e-17]]

Layer 0 weights rmse 4.279844320860524e-34

output [array([0.    , 0.    , 1.4183]), array

In [390]:
run_test_case_reference_mode(f"tc_b/{test_cases[4]}")

Running test case for tc_b/relu_b.json


1/1 [==============================] - 0s 240ms/step
raw weight
[array([[ 0.30110002,  0.5095    ,  0.4335    ],
       [-0.49633333, -0.9683333 ,  0.43033332]], dtype=float32), array([-0.20366667,  0.16833334,  0.96166664], dtype=float32)]
 weight shape (2, 3) bias shape (3,)
result weight for layer-0
[[-0.20366667  0.16833334  0.96166664]
 [ 0.30110002  0.50950003  0.43349999]
 [-0.49633333 -0.9683333   0.43033332]]
expected weight for layer-0
[[-0.211   0.105   0.885 ]
 [ 0.3033  0.5285  0.3005]
 [-0.489  -0.905   0.291 ]]

Layer 0 weights rmse 0.005719573156393852

output 
[[0.         0.         1.7394333 ]
 [0.20233665 0.9838166  0.40128332]]
expect [[1.  0.1 0.1]
 [0.1 0.1 1. ]]
Testing result error 0.8079679635266404


### Sigmoid.json
Testcase untuk sigmoid.json

In [391]:
run_test_case_mode(f"tc_b/{test_cases[5]}")

Running test case for tc_b/sigmoid.json


Initial weight for layer 0
[[0.3 0.1]
 [0.2 0.6]
 [0.8 0.3]]
/* Layer-1 (sigmoid) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.8)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (0.3)






Training data input 2 rows
Begin training model


Epoch 0
training loss 0.47078073489758887


Epoch 1
training loss 0.4687675781198633


Epoch 2
training loss 0.4667706588834645


Epoch 3
training loss 0.4647899555600068


Epoch 4
training loss 0.4628254366344452


Epoch 5
training loss 0.46087706106867526


Epoch 6
training loss 0.45894477867098016


Epoch 7
training loss 0.4570285304700267


Epoch 8
training loss 0.4551282490921642


Epoch 9
training loss 0.4532438591408193



Stop reason by max_iteration expected max_iteration

result weight for layer-0
[[0.23291176 0.06015346]
 [0.12884088 0.64849474]
 [0.837615   0.23158199]]
expected weight for layer-0
[[0.2329 0.0601]
 [0.1288 0.6484]
 [0.8376 0.2315]]
abs wei

In [392]:
run_test_case_reference_mode(f"tc_b/{test_cases[5]}")

Running test case for tc_b/sigmoid.json


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.2364
Epoch 2/10
1/1 [==============================] - 0s 58ms/step - loss: 0.2359
Epoch 3/10
1/1 [==============================] - 0s 36ms/step - loss: 0.2354
Epoch 4/10
1/1 [==============================] - 0s 42ms/step - loss: 0.2349
Epoch 5/10
1/1 [==============================] - 0s 20ms/step - loss: 0.2344
Epoch 6/10
1/1 [==============================] - 0s 19ms/step - loss: 0.2339
Epoch 7/10
1/1 [==============================] - 0s 19ms/step - loss: 0.2334
Epoch 8/10
1/1 [==============================] - 0s 20ms/step - loss: 0.2329
Epoch 9/10
1/1 [==============================] - 0s 79ms/step - loss: 0.2324
Epoch 10/10
1/1 [==============================] - 0s 362ms/step
raw weight
[array([[0.16422285, 0.62417907],
       [0.81860363, 0.26559675]], dtype=float32), array([0.2656529 , 0.07955156], dtype=float32)]
 weight shape (2, 2) bias shape (2,)
result weight

### Softmax.json
Testcase untuk softmax.json

In [393]:
run_test_case_mode(f"tc_b/{test_cases[6]}")

Running test case for tc_b/softmax.json


Initial weight for layer 0
[[ 0.1  0.9 -0.1]
 [-0.2  0.8  0.2]
 [ 0.3 -0.7  0.3]
 [ 0.4  0.6 -0.4]
 [ 0.5  0.5  0.5]
 [-0.6  0.4  0.6]
 [-0.7 -0.3  0.7]
 [ 0.8  0.2 -0.8]
 [ 0.9 -0.1  0. ]]
/* Layer-1 (softmax) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.3)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-1 (0.4)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-1 (0.5)
Input Layer Neuron-4 -> Hidden Layer-1 Neuron-1 (-0.6)
Input Layer Neuron-5 -> Hidden Layer-1 Neuron-1 (-0.7)
Input Layer Neuron-6 -> Hidden Layer-1 Neuron-1 (0.8)
Input Layer Neuron-7 -> Hidden Layer-1 Neuron-1 (0.9)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (-0.7)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-2 (0.6)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-2 (0.5)
Input Layer Neuron-4 -> Hidden Layer-1 Neuron-2 (0.4)
Input Layer Neuron-5 -> Hidden Layer-1 Neuron-2 (-0.3)
Input Layer Neuron-6 -> Hidden Layer-1 Neuron-2 (0.2)
Input

In [394]:
run_test_case_reference_mode(f"tc_b/{test_cases[6]}")

Running test case for tc_b/softmax.json


Epoch 1/10
3/3 [==============================] - 2s 19ms/step - loss: 0.4418
Epoch 2/10
3/3 [==============================] - 0s 10ms/step - loss: 0.4312
Epoch 3/10
3/3 [==============================] - 0s 10ms/step - loss: 0.4200
Epoch 4/10
3/3 [==============================] - 0s 10ms/step - loss: 0.4086
Epoch 5/10
3/3 [==============================] - 0s 19ms/step - loss: 0.3970
Epoch 6/10
3/3 [==============================] - 0s 10ms/step - loss: 0.3855
Epoch 7/10
3/3 [==============================] - 0s 10ms/step - loss: 0.3744
Epoch 8/10
3/3 [==============================] - 0s 10ms/step - loss: 0.3639
Epoch 9/10
3/3 [==============================] - 0s 10ms/step - loss: 0.3541
Epoch 10/10
1/1 [==============================] - 0s 239ms/step
raw weight
[array([[-0.1742265 ,  0.74874806,  0.2254784 ],
       [ 0.32756075, -0.7609455 ,  0.3333849 ],
       [ 0.40776226,  0.5877591 , -0.39552128],
       [ 0.4676497 ,  0.49711534,  

### softmax_two_layer.json
Testcase untuk softmax_two_layer.json

In [395]:
run_test_case_mode(f"tc_b/{test_cases[7]}")

Running test case for tc_b/softmax_two_layer.json


Initial weight for layer 0
[[ 0.1 -0.1  0.1 -0.1]
 [-0.1  0.1 -0.1  0.1]
 [ 0.1  0.1 -0.1 -0.1]]
Initial weight for layer 1
[[ 0.12 -0.1 ]
 [-0.12  0.1 ]
 [ 0.12 -0.1 ]
 [-0.12  0.1 ]
 [ 0.02  0.  ]]
/* Layer-1 (relu) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.1)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (0.1)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (-0.1)



/* Neuron-4 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-4 (-0.1)






/* Layer-2 (softmax) */
/* Neuron-1 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-1 (0.12)
Hidden Layer-1 Neuron-2 -> Hidden Layer-2 Neuron-1 (-0.12)
Hidden Layer-1 Neuron-3 -> Hidden Layer-2 Neuron-1 (0.02)



/* Neuron-2 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-2 (-0.1)
Hidden Layer-1 Neuron-2 -> Hidden Layer-2 Neuron-2 (0.1)






Training data input 8 rows
Begin training model


Epoch 0
training loss 0.6744156

In [396]:
run_test_case_reference_mode(f"tc_b/{test_cases[7]}")

Running test case for tc_b/softmax_two_layer.json


Epoch 1/200
8/8 [==============================] - 3s 12ms/step - loss: 0.2682
Epoch 2/200
8/8 [==============================] - 0s 23ms/step - loss: 0.2596
Epoch 3/200
8/8 [==============================] - 0s 14ms/step - loss: 0.2554
Epoch 4/200
8/8 [==============================] - 0s 43ms/step - loss: 0.2516
Epoch 5/200
8/8 [==============================] - 0s 31ms/step - loss: 0.2499
Epoch 6/200
8/8 [==============================] - 0s 20ms/step - loss: 0.2471
Epoch 7/200
8/8 [==============================] - 0s 9ms/step - loss: 0.2438
Epoch 8/200
8/8 [==============================] - 0s 14ms/step - loss: 0.2405
Epoch 9/200
8/8 [==============================] - 0s 17ms/step - loss: 0.2365
Epoch 10/200
8/8 [==============================] - 0s 9ms/step - loss: 0.2336
Epoch 11/200
8/8 [==============================] - 0s 12ms/step - loss: 0.2271
Epoch 12/200
8/8 [==============================] - 0s 14ms/step - loss: 0.2179

## Konfigurasi Dataset Iris
Berikut merupakan konfigurasi dataset Iris

### Preprocessing Dataset

In [397]:
# Fetch and process dataset

from sklearn.model_selection import train_test_split
# Testcase Iris
import pandas as pd

df = pd.read_csv("iris.csv")

dict_target = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica' : 2 }

x = df.iloc[:,1:-1]
x=(x-x.mean())/x.std()
y = df.iloc[:, -1].replace(dict_target)

target_count = len(y.unique())

def generate_target(target: int):
    target_array = np.zeros((target_count,), dtype=float)
    target_array[target] = 1.0
    return target_array

y_processed = np.zeros(shape=(len(y), target_count), dtype=float)

for i, element in enumerate(y):
    y_processed[i] = generate_target(element)

x_train, x_test, y_train, y_test = train_test_split(x.to_numpy(), y_processed, test_size = 0.2, random_state=0)

# print_debug(x_train)

input_size = len(x_train[0])

### Konfigurasi Model

In [398]:
# Set configs for the model
learning_rate = 0.1
batch_size = 1
max_iteration = 15
error_threshold = 0.05

### Training Model

In [399]:
# Train and save model

model = Model()

layers = [
    {
        "number_of_neurons": 3,
        "activation_function": "linear",
        "input_size": input_size
    },
    {
        "number_of_neurons": target_count,
        "activation_function": "sigmoid"
    }
]


for i, layer in enumerate(layers):
        if i == 0:
            model.add(
                Layer(
                    n_neurons=layer["number_of_neurons"],
                    activation_function=layer["activation_function"],
                    input_size=layer["input_size"]
                )
            )
        else:
            model.add(
                Layer(
                    n_neurons=layer["number_of_neurons"],
                    activation_function=layer["activation_function"],
                )
            )


model.compile()

# The initial weight isn't manually set because it's already set randomly during the initialization of the layer

model.visualize()

model.fit(
        x=x_train,
        y=y_train,
        learning_rate=learning_rate,
        epochs=max_iteration,
        batch_size=batch_size,
        error_threshold=error_threshold
    )

saved_filename = "model_iris.json"

saved_model = SavedModel.from_model(model)
saved_model.save_to(saved_filename)

# Lalu lakukan load kembali trained model (jadi ga perlu fit lagi)
# Model yang di-load ini langsung bisa digunakan untuk predict menggunakan data test

/* Layer-1 (linear) */
/* Neuron-1 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-1 (0.24875314351995803)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-1 (0.5761573344178369)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-1 (0.592041931271839)



/* Neuron-2 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-2 (0.952749011516985)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-2 (0.44712537861762736)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-2 (0.8464086724711278)



/* Neuron-3 */
Input Layer Neuron-1 -> Hidden Layer-1 Neuron-3 (0.8137978197024772)
Input Layer Neuron-2 -> Hidden Layer-1 Neuron-3 (0.39650574084698464)
Input Layer Neuron-3 -> Hidden Layer-1 Neuron-3 (0.8811031971111616)






/* Layer-2 (sigmoid) */
/* Neuron-1 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-1 (0.6925315900777659)
Hidden Layer-1 Neuron-2 -> Hidden Layer-2 Neuron-1 (0.7252542798196405)



/* Neuron-2 */
Hidden Layer-1 Neuron-1 -> Hidden Layer-2 Neuron-2 (0.6439901992296374)
Hidden Layer-1 Neuron-2 -

### Load Model dan Predict

In [400]:
# Load model and use it to make predictions
saved_model = SavedModel.from_file(saved_filename)
iris_model = Model.from_saved_model(saved_model)

results = iris_model.predict_batch(x_test)
for result in results:
    max_element_idx = np.argmax(result)
    for i in range(len(result)):
        if i != max_element_idx:
            result[i] = 0
        else:
            result[i] = 1

counter = 0
for i, test in enumerate(y_test):
    print(f"y test: {test} \t y prediction: {results[i]} \t {np.argmax(test) == np.argmax(results[i])}")
    if np.argmax(test) == np.argmax(results[i]):
        counter+=1
    
print(f"accuracy: {counter / len(y_test) * 100}%")

y test: [0. 0. 1.] 	 y prediction: [0. 0. 1.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [1. 0. 0.] 	 y prediction: [1. 0. 0.] 	 True
y test: [0. 0. 1.] 	 y prediction: [0. 0. 1.] 	 True
y test: [1. 0. 0.] 	 y prediction: [1. 0. 0.] 	 True
y test: [0. 0. 1.] 	 y prediction: [0. 0. 1.] 	 True
y test: [1. 0. 0.] 	 y prediction: [1. 0. 0.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [0. 0. 1.] 	 y prediction: [0. 1. 0.] 	 False
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [1. 0. 0.] 	 y prediction: [1. 0. 0.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [0. 1. 0.] 	 y prediction: [0. 1. 0.] 	 True
y test: [1. 0. 0.] 	 y prediction: [1. 0. 0.]

# lama

In [401]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import json

def encode_target(data, target_name):
    label_encoder = LabelEncoder()
    data['encoded_target'] = label_encoder.fit_transform(data[target_name])
    data_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    return data, data_mapping

def generate_weight(layers, input_size):
    initial_weights = []
    previous_size = input_size
    for layer in layers:
        layer_weights = np.random.randn(layer['number_of_neurons'], previous_size).tolist()
        initial_weights.append(layer_weights)
        previous_size = layer['number_of_neurons']
    return initial_weights

def generate_layers(layers_info):
    layers = []
    for layer_info in layers_info:
        layers.append({
            "number_of_neurons": layer_info["number_of_neurons"],
            "activation_function": layer_info["activation_function"]
        })
    return layers

def convert_to_json(data, input_columns, layers_info, target_column, learning_rate, batch_size, max_iteration, error_threshold):
    input_size = len(input_columns)
    layers = generate_layers(layers_info)
    initial_weights = generate_weight(layers, input_size)
    
    data, _ = encode_target(data, target_column)
    
    json_structure = {
        "case": {
            "model": {
                "input_size": input_size,
                "layers": layers
            },
            "input": data[input_columns].values.tolist(),
            "initial_weights": initial_weights,
            "target": data['encoded_target'].values.reshape(-1, 1).tolist(),
            "learning_parameters": {
                "learning_rate": learning_rate,
                "batch_size": batch_size,
                "max_iteration": max_iteration,
                "error_threshold": error_threshold
            }
        },
        "expect": {
            "stopped_by": "max_iteration",
            "final_weights": []  # This needs actual implementation to compute final weights
        }
    }
    
    return json_structure

In [402]:
input_columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
layers_info = [
    {"number_of_neurons": 5, "activation_function": "relu"},
    {"number_of_neurons": 3, "activation_function": "sigmoid"}
]
learning_rate = 0.1
batch_size = 2
max_iteration = 1
error_threshold = 0.0

data = pd.read_csv('iris.csv')

model_config = convert_to_json(data, input_columns, layers_info, 'Species', learning_rate, batch_size, max_iteration, error_threshold)
print(json.dumps(model_config, indent=4))

{
    "case": {
        "model": {
            "input_size": 4,
            "layers": [
                {
                    "number_of_neurons": 5,
                    "activation_function": "relu"
                },
                {
                    "number_of_neurons": 3,
                    "activation_function": "sigmoid"
                }
            ]
        },
        "input": [
            [
                5.1,
                3.5,
                1.4,
                0.2
            ],
            [
                4.9,
                3.0,
                1.4,
                0.2
            ],
            [
                4.7,
                3.2,
                1.3,
                0.2
            ],
            [
                4.6,
                3.1,
                1.5,
                0.2
            ],
            [
                5.0,
                3.6,
                1.4,
                0.2
            ],
            [
                5.4,
         

In [403]:
import numpy as np

# Fungsi Aktivasi Turunan

activation_functions = {
        "linear": lambda x: x,
        "relu": lambda x: np.maximum(0, x),
        "sigmoid": lambda x: 1/(1+np.exp(-x)),
        "softmax": lambda x: np.exp(x)/np.sum(np.exp(x))
    }

def softmax_derivation(x: np.array, i: int):
    result = np.array(np.vectorize(lambda y: y-1)(x))
    result[i] = x[i]
    return result

derived_activation_functions = {
        "linear": lambda x: np.ones(len(x)),
        "relu": lambda x: np.array(np.vectorize(lambda y: 1 if y >= 0 else 0)(x)),
        "sigmoid": lambda x: activation_functions["sigmoid"](x) * (1 - activation_functions["sigmoid"](x)),
        "softmax": softmax_derivation
}

# Fungsi Loss
def sse(o: np.array, t: np.array):
    return 0.5*np.sum((t-o)**2)

def softmax_loss(o: np.array, i: int):
    return o[i]

notusedloss_functions = {
        "linear": sse,
        "relu": sse,
        "sigmoid": sse,
        "softmax": softmax_loss
}

In [404]:
import json
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class TestCase:
    expected_output: List[List[float]]
    input_size: int
    layers: List[Tuple[int, str]]
    weights: List[np.array]
    input_data: np.array
    max_sse: float

    def print_info(self):
        print(f"Layers\n{self.layers}\n\nWeights{self.weights}\n\n")
        print(f"Input size\n{self.input_size}\n\nInput data\n{self.input_data}\n\n")
        print(f"Expected output\n{self.expected_output}\n\nMax SSE\n{self.max_sse}")

    @staticmethod
    def from_file(filename: str):
        json_read = json.load(open(filename, "r"))
        max_sse = json_read["expect"]["max_sse"]
        output = json_read["expect"]["output"]
        input_size = json_read["case"]["model"]["input_size"]
        layers_raw = json_read["case"]["model"]["layers"]
        layers = []
        for element in layers_raw:
            layers.append(tuple([element["number_of_neurons"], element["activation_function"]]))

        weights_raw = json_read["case"]["weights"]
        weights = []

        for weight_raw in weights_raw:
            weight = []
            for j in weight_raw:
                weight.append(np.array(j))
            weights.append(np.array(weight).T)

        input_data = np.array(json_read["case"]["input"])

        return TestCase(
            expected_output=output,
            input_size=input_size,
            layers=layers,
            weights=weights,
            input_data=input_data,
            max_sse=max_sse
        )

- learning_rate (during fit)
- activation (per layer)
- epochs (during fit)
- batch_size (during fit)
- threshold (during fit)

In [405]:
import graphviz as gv

class Neuron:
    weight: np.array
    d_weight: List[np.array]
    result: float

    def __init__(self, weight: np.array):
        self.weight = weight
        self.d_weight = []

    """
    return bias + wt•x
    """
    def compute(self, input_data: np.array):
        self.result = self.weight[0] + np.dot(input_data, self.weight[1:])
        return self.result
    
    def init_d_weight(self, batch_size: int):
        self.d_weight = [np.arange(len(self.weight)) for _ in range(batch_size)]

    def _reset_d_weight(self):
        self.d_weight = []

    def persist_d_weight(self):
        for i in range(len(self.weight)):
            self.weight[i] += np.sum(self.d_weight[i]) / len(self.d_weight[i])
        self.reset_d_weight()

class Layer:
    neurons: List[Neuron]
    raw_result : np.array
    result : np.array
    # As long as the amount of neuron in the layer before it
    do_c_do_a: List[float]
    result_derivative : np.array
    activation_function_name: str
    activation_functions = {
        "linear": lambda x: x,
        "relu": lambda x: np.maximum(0, x),
        "sigmoid": lambda x: 1/(1+np.exp(-x)),
        "softmax": lambda x: np.exp(x)/np.sum(np.exp(x))
    }

    activation_functions_derivative = {
        "linear": lambda x: np.ones(len(x)),
        "relu": lambda x: np.array(np.vectorize(lambda y: 1 if y >= 0 else 0)(x)),
        "sigmoid": lambda x: activation_functions["sigmoid"](x) * (1 - activation_functions["sigmoid"](x)),
        "softmax": softmax_derivation
    }

    def __init__(self, n_neurons: int, activation_function_name: str, weights: List[np.array]):
        self.neurons = []
        # Initialize neurons in the layer
        for i in range(0, n_neurons):
            self.neurons.append(Neuron(weights[i]))
        self.activation_function_name = activation_function_name

    def predict(self, input_data: np.array) -> np.array:
        self.raw_result = np.array([neuron.compute(input_data) for neuron in self.neurons])
        self.result = self.activation_functions[self.activation_function_name](self.raw_result)
        return self.result
    
    def predict_derivative(self):
        self.result_derivative = self.activation_functions_derivative[self.activation_function_name](self.raw_result)
        return self.result_derivative

    def persist_d_weight(self):
        for neuron in self.neurons:
            neuron.persist_d_weight()
    
    def init_d_weight(self, batch_size: int):
        for neuron in self.neurons:
            neuron.init_d_weight(batch_size)
        
class Model:
    input_size: int
    layers: List[Layer]
    learning_rate: int

    @staticmethod
    def from_test_case(test_case: TestCase):
        return Model(test_case.input_size, test_case.layers, test_case.weights)

    def __init__(self, input_size: int, layers_attr: List[Tuple[int, str]], weights: List[np.array]):
        self.input_size = input_size
        # Initialize layers
        self.layers = []
        for i in range(0, len(layers_attr)):
            layer: Layer = Layer(layers_attr[i][0], layers_attr[i][1], weights[i])
            self.layers.append(layer)
    
    def init_d_weight(self, batch_size: int):
        for layer in self.layers:
            layer.init_d_weight(batch_size)

    def persist_d_weight(self):
        for layer in self.layers:
            layer.persist_d_weight()

    def _predict(self, input_data: np.array) -> np.array:
        layer: Layer = self.layers[0]
        temp_array = layer.predict(input_data)
        for i in range(1, len(self.layers)):
            layer: Layer = self.layers[i]
            temp_array = layer.predict(temp_array)
        
        return temp_array
        
    def predict_batch(self, input_data: List[np.array]) -> List[np.array]:
        # Batch output
        final_output = []
        for data in input_data:
            final_output.append(self._predict(data))
        return final_output

    def visualize(self):
        graph = gv.Digraph(filename="./output/graph.gv")
        for i in range(self.input_size):
            graph.node(f"IN{i}", f"Input Neuron-{i + 1}")
        graph.render()

    def visualize_print(self):
        for i, layer in enumerate(self.layers):
            print(f"/* Hidden Layer-{i + 1} ({layer.activation_function_name}) */")
            for j, neuron in enumerate(layer.neurons):
                print(f"/* Neuron-{j + 1} */")
                if i == 0:
                    source_layer = "Input Layer"
                else:
                    source_layer = f"Hidden Layer-{i}"
                for k, w in enumerate(neuron.weight):
                    if k == 0:
                        continue
                    if w != 0:
                        print(f"{source_layer} Neuron-{k} -> Hidden Layer-{i + 1} Neuron-{j+1} ({w})")
                print("\n\n")
            print("\n\n")

    def _generate_mini_batches(self, input_data: List[np.array], expected_output: List[np.array], batch_size: int):
        start_index = 0
        input_size = len(input_data)
        mini_batches_input = []
        mini_batches_output = []
        while start_index < input_size:
            end_index = min(start_index + batch_size, input_size)
            mini_batches_input.append(input_data[start_index:end_index])
            mini_batches_output.append(expected_output[start_index:end_index])
            start_index += batch_size
        
        return mini_batches_input, mini_batches_output
    
    def _backpropagate(self,expected_output: np.array, nth_batch: int):

        for i in range(len(self.layers) - 1, 1, -1):
            layer_current = self.layers[i]
            layer_previous = None if i == 0 else self.layers[i - 1]
            layer_next = None if i == len(self.layers) - 1 else self.layers[i + 1]

            if layer_next is not None :
                # Update weight of hidden layer

                # Calculate do_c_do_a to be used in calculating do_c_do_w
                # do_c_do_a of output layer has a special value, calculated separately below
                # this is for when layer next is hidden layer
                if i != len(self.layers) - 2:
                    layer_next.predict_derivative()
                    for nth_neuron, neuron_current in enumerate(layer_next.neurons):
                        layer_next.do_c_do_a[nth_neuron] = np.sum(neuron_current.weight  * layer_next.result_derivative[nth_neuron] * self.layers[i + 2].do_c_do_a[nth_neuron])
                
                # Calculate do_c_do_w
                for nth_neuron, neuron_current in enumerate(self.layers[i].neurons):
                    for nth_weight, weight_current in enumerate(neuron_current.weight):
                        do_c_do_w = layer_previous.result[nth_weight] * layer_current.result_derivative[nth_neuron] * layer_next.do_c_do_a[nth_neuron]
                        neuron_current.d_weight[nth_batch][nth_weight] = self.learning_rate * do_c_do_w
            else:
                # Update weight of output layer
                layer_current.do_c_do_a = np.subtract(layer_current.result, expected_output)
                for nth_neuron, neuron_current in enumerate(layer_current.neurons):
                    for nth_weight, weight_current in enumerate(neuron_current.weight):
                        neuron_previous_activation = layer_previous.result[nth_weight]
                        do_c_do_w = neuron_previous_activation * layer_current.result_derivative[nth_neuron] * layer_current.do_c_do_a[nth_neuron]
                
                        neuron_current.d_weight[nth_batch][nth_weight] = self.learning_rate * do_c_do_w * neuron_previous_activation
            
    # Update weight for batches
    # run the backpropagation for all the data in the current batch
    # after it is done, we aggregate the weight changes from the _backpropagate function, then we apply to the model
    def backpropagate(self, batch_result_expected: np.array):            
        for i, result_expected in enumerate(batch_result_expected):
            self._backpropagate(result_expected, i)
        self.persist_d_weight()

    def train(self, input_data: List[np.array], expected_output: List[np.array], error_threshold: float, max_iteration: int, batch_size: int, learning_rate: int):
        self.learning_rate = learning_rate
        self.init_d_weight(batch_size)
        # Epoch
        for _ in range(max_iteration):
            for mini_batch_input, mini_batch_output in self._generate_mini_batches(input_data, expected_output):
                # Get the output first
                self.predict_batch(mini_batch_input)
                self.backpropagate(mini_batch_output)

                # TODO: Define error
                error = 0
                if (error < error_threshold): 
                    break
        pass

In [406]:
def calculate_sse(output_data: List[np.array], expected_data: List[np.array]):
    if len(output_data) != len(expected_data):
        raise ValueError("Output and Expected Data length doesn't match.")
    
    sses = []

    for output, expected in zip(output_data, expected_data):
        delta = output - expected
        squared_delta = delta ** 2
        sses.append(np.sum(squared_delta))

    return sses

def evaluate_result(output_data: List[np.array], output_reference_data: np.array, expected_data: List[np.array], max_sse: float):
    sses = calculate_sse(output_data, expected_data)

    for i in range(len(output_data)):
        predicted = output_data[i]
        predicted_reference = output_reference_data[i]
        expected = expected_data[i]
        sse = sses[i]

        print(f"Prediction result:\n{predicted}")
        print(f"Prediction reference result:\n{predicted_reference}")
        print(f"Expected output:\n{expected}")

        print(f"sse {sse}\tmax sse{max_sse}")
        print(f"Is below error? {sse < max_sse}")

Setelah dilakukan Feed Forward yang memprediksi hasil dari sebuah data, dilakukan Backpropagation untuk mengupdate weight dan juga aktivasi dari neuron dalam model.

Untuk mengupdate weight, digunakan persamaan

\begin{equation}
w_{jk} = w_{jk} - \Delta w_{jk}
\end{equation}

\begin{equation}
\Delta w_{jk} = -n \frac{\partial C}{\partial w^{(l)}_{jk}}
\end{equation}

\begin{equation}
w_{jk} = w_{jk} + n \frac{\partial C}{\partial w^{(l)}_{jk}}
\end{equation}

dimana

\begin{equation}
\frac{\partial C}{\partial w^{(l)}_{jk}} = \frac{\partial z^{(l)}\_{j}}{\partial w^{(l)}\_{jk}} \frac{\partial a^{(l)}\_{j}}{\partial z^{(l)}\_j} \frac{\partial C}{\partial a^{(l)}_j}
\end{equation}

dan

\begin{equation}
z{^{(l)}_j} = \Sigma{^n\_{k=0}} w^{(l)}\_{jk} a^{(l -1)}_k + b_j^{(l)} \rightarrow \\ \frac{\partial z^{(l)}\_{j}}{\partial w^{(l)}\_{jk}} = a_k^{(l-1)}
\end{equation}

\begin{equation}
a{^{(l)}_j} = \sigma ( z{^{(l)}_j} ) \rightarrow \\ \frac{\partial a^{(l)}\_{j}}{\partial z^{(l)}\_j} = \sigma '(z_j^{(l)})
\end{equation}

\begin{equation}$
\frac{\partial C}{\partial a^{(l)}_j}
 = \left\{ 
  \begin{array}{ c l }
    \Sigma{_{j=0}^{n_{(l+1)}-1}} w^{(l+1)}_{jk} \sigma '(z_j^{(l+1)}) \frac{\partial C}{\partial a^{(l+1)}_j}  & \quad \textrm{for hidden layer}\\
    (a_j^{(l)} - y_j)                 & \quad \textrm{for output layer}
  \end{array}
\right.$
\end{equation}

sehingga didapatkan

\begin{equation}
\frac{\partial C}{\partial w^{(l)}_{jk}} = a_k^{(l-1)} \sigma '(z_j^{(l)}) \frac{\partial C}{\partial a^{(l)}_j}
\end{equation}

maka untuk setiap layer, dapat dihitung 
\begin{equation} 
\frac{\partial C}{\partial a^{(l)}_j}
\end{equation}

yang akan digunakan untuk menghitung
\begin{equation} 
\frac{\partial C}{\partial w^{(l)}_{jk}}
\end{equation}
yang akan digunakan untuk menghitung bobot, dimana bobot akan di-persist setiap mini-batch

# Iris

In [407]:
import tensorflow as tf
from typing import List
from dataclasses import dataclass
from tensorflow.keras.layers import Dense, Input
from keras.models import Sequential
from keras.optimizers import SGD
from keras.losses import MeanSquaredError
from tensorflow.keras.models import Model

import numpy as np
import json

In [408]:
# Setup

# np.random.seed(0)

# DEBUG_MODE: bool = True
# TESTCASE_MODE: bool = True
# TESTCASE_REFERENCE_MODE: bool = True

# def run_test_case_mode(filename):
#     if TESTCASE_MODE:
#         run_test_case(filename)

# def run_test_case_reference_mode(filename):
#     if TESTCASE_MODE:
#         run_test_on_reference(filename)

# def print_debug(value: str):
#     if DEBUG_MODE:
#         print(value)